# Linking named entities
Filip Gregora

In [97]:
import pandas as pd

data = pd.read_csv("data/NER_entities.csv")

In [98]:
data.head(10)

,label,text
0,symptom,jemný fibrózní proužek
1,procedura,neoadjuvantní CHT
2,medikace,Novalgin
3,symptom,Označena SLU v levé axile.
4,procedura,st.p. totální ME + SNB vlevo
5,medikace,NOVALGIN
6,procedura,Založení TE l.sin
7,procedura,Cytostatika
8,NE symptom,"přiměřené echogenity,"
9,NE symptom,nezvětšena


## Data exploration
At the beginning we want to explore data.

- We can see that somewhere there is big letter at the beginning (but be carefull when whole first word is written in upper case)
- Somewhere at the end is interpuncion
- There are lots of duplicates
- the length of text is variable and the longest has 20 words (this can be problem in the future)


In [99]:
len(data[data.duplicated()])

2588

In [100]:
from string import punctuation

def clean_db(db):
    db_copy = db.copy()
    db_copy["text"] = db_copy["text"].apply(lambda x: x.strip(" " + "".join(punctuation)))
    db_copy["text"] = db_copy["text"].apply(lambda x: x[0].lower() + x[1:] if x[1].islower() else x)
    db_copy["text"] = db_copy["text"].apply(lambda x: " ".join(x.split())) #to replace multiple whitespaces with one
    db_copy["text"] = db_copy["text"].drop_duplicates()
    return db_copy.dropna()

data = clean_db(data)
data.head(10)

,label,text
0,symptom,jemný fibrózní proužek
1,procedura,neoadjuvantní CHT
2,medikace,novalgin
3,symptom,označena SLU v levé axile
4,procedura,st.p. totální ME + SNB vlevo
5,medikace,NOVALGIN
6,procedura,založení TE l.sin
7,procedura,cytostatika
8,NE symptom,přiměřené echogenity
9,NE symptom,nezvětšena


In [101]:
import math
def comb_sum(j):
    sum = 0
    for i in range(j, 0, -1):
        sum += math.comb(j,i)

    return sum

for i in range(1, 21):
    print(i, comb_sum(i), sep = ": ", end = " | ")
    
lenght_data = data["text"].apply(lambda x: len(x.split(" ")))
len(lenght_data[lenght_data >= 7])

1: 1 | 2: 3 | 3: 7 | 4: 15 | 5: 31 | 6: 63 | 7: 127 | 8: 255 | 9: 511 | 10: 1023 | 11: 2047 | 12: 4095 | 13: 8191 | 14: 16383 | 15: 32767 | 16: 65535 | 17: 131071 | 18: 262143 | 19: 524287 | 20: 1048575 | 

140

In [102]:
symptom = data[data.label == "symptom"].drop(["label"], axis=1)
procedura = data[data.label == "procedura"].drop(["label"], axis=1)
medikace = data[data.label == "medikace"].drop(["label"], axis=1)
ne_symptom = data[data.label == "NE symptom"].drop(["label"], axis=1)
os_anamneza = data[data.label == "osobní anamnéza"].drop(["label"], axis=1)
ne_os_anamneza = data[data.label == "NE osobní anamnéza"].drop(["label"], axis=1)
ne_medikace = data[data.label == "NE medikace"].drop(["label"], axis=1)

In [103]:
assert(len(os_anamneza) + len(symptom) + len(procedura) + len(medikace) + len(ne_symptom) + len(ne_os_anamneza) + len(ne_medikace) \
            == len(data))

## Linking to international MASH through NIH
Mash is international medical databaze: https://uts.nlm.nih.gov/uts/.

I tried search all combinations of words from text in databaze. The longer combinations have higher priority. 

There is one big problem, the complexity grows exponentially with the lenght of the words (in the worst case for lenght of 20 we have to try around 10^6 combinations). My solution for this problem is go from bottom up, start with lenght 1 and continue only with combinations which success.


In [104]:
# Do not search in databaze if it number or it is too short (shorter than 2)
def filter_short(string):
    return len(string) < 2 or string.isdigit()
    
    
def print_stats(data_list):
    empty = len(data_list[data_list.apply(lambda x: len(x) == 0)])
    print(f"Number of empty: {empty} ({empty / len(data_list) * 100} %)")

    number_of_matches = data_list.apply(lambda x: len(x))
    print(f"Mean from number of matches: {number_of_matches.mean()}")
    print(f"Median from number of matches: {number_of_matches.median()}")
    print(f"Maximal of matches: {number_of_matches.max()}")
    
    
def from_string_to_list(string):
    result = []
    for j in string.strip("[]()").split("), ("):
        if len(j) == 0:
            continue
        result.append(tuple(j.strip("'").split("', '")))
                
    return result

In [105]:
import requests
import json
from itertools import combinations

def mash_search_basic(string):
    splitted_input = (string.split(" "))
    result = []
    for j in range(len(splitted_input), 0, -1):
        for string in combinations(splitted_input, j): 
            if filter_short(" ".join(string)):
                continue
                
            path = 'https://uts-ws.nlm.nih.gov/rest/search/current'
            query = {
                     'string': " ".join(string),
                     'apiKey':'6a290909-c0d8-4db9-b531-7387929b334e',
            }
            res = requests.get(path, params=query)

            if res.status_code <= 200:
                data = json.loads(res.text)
                for j in data["result"]["results"]:
                    result.append((j["ui"], j["name"]))
            else:
                print(res.status_code, res.text)
        
        if len(result) != 0:
            break
                        
    return result
        
    
def search_with_inclusion(string, func, output_state = False):
    if (output_state):
        global count
        count += 1
        if count % 100 == 0:
            print(count)
    
    splitted_input = (string.split(" "))
    result = []
    last_result = []
    lenght = len(splitted_input)
    
    for j in range(1, lenght + 1):
        splitted_dict={}
        for elem in splitted_input:
            splitted_dict[j] = False
                    
        for words in combinations(splitted_input, j):
            data = func(" ".join(words))
            if len(data) != 0:
                for j in words:
                    splitted_dict[j] = True
                if filter_short(" ".join(words)):
                    continue
                result.append(data)

        splitted_input = [j for j, i in splitted_dict.items() if i]
        if len(splitted_input) == 0:
            break
        else:
            last_result, result = result, []
        
    temp = []
    for j in last_result:
        temp += list(enumerate(j))
    return [j for (i, j) in sorted(temp)]
    
    
def mash_search(string):
    path = 'https://uts-ws.nlm.nih.gov/rest/search/current'
    query = {
             'string': string,
             'apiKey':'6a290909-c0d8-4db9-b531-7387929b334e',
    }
    res = requests.get(path, params=query)

    if res.status_code <= 200:
        data = json.loads(res.text)          
        return [(j["ui"], j["name"]) for j in data["result"]["results"]]
    else:
        print(res.status_code, res.text)
        return []
    

def search_db(db, func):
    db = db.copy()
    db["search"] = db["text"].apply(func)
    return db 
    
    
def search_db_mash(db):
    db = db.copy()
    db["search"] = db["text"].apply(mash_search_basic)
    return db    


def search_db_mash_optimized(db):
    db = db.copy()
    db["search"] = db["text"].apply(lambda x: search_with_inclusion(x, mash_search))
    return db

print(mash_search_basic('bolesti patní ostruhy vlevo'))

[('C0149756', 'Fasciitis, Plantar')]


In [106]:
import os
if os.path.isfile("saved_search/os_anamneza_mash.csv"):
    os_anamneza_mash = pd.read_csv("saved_search/os_anamneza_mash.csv")
    os_anamneza_mash.index = os_anamneza_mash["Unnamed: 0"]
    os_anamneza_mash.drop(["Unnamed: 0"], axis=1, inplace=True)
    os_anamneza_mash["search"] = os_anamneza_mash["search"].apply(from_string_to_list)
else:
    os_anamneza_mash = search_db_mash_optimized(os_anamneza)
    os_anamneza_mash.to_csv("saved_search/os_anamneza_mash.csv")
    
if os.path.isfile("saved_search/ne_os_anamneza_mash.csv"):
    ne_os_anamneza_mash = pd.read_csv("saved_search/ne_os_anamneza_mash.csv")
    ne_os_anamneza_mash.index = ne_os_anamneza_mash["Unnamed: 0"]
    ne_os_anamneza_mash.drop(["Unnamed: 0"], axis=1, inplace=True)
    ne_os_anamneza_mash["search"] = ne_os_anamneza_mash["search"].apply(from_string_to_list)
else:
    ne_os_anamneza_mash = search_db_mash_optimized(ne_os_anamneza)
    ne_os_anamneza_mash.to_csv("saved_search/ne_os_anamneza_mash.csv")    

if os.path.isfile("saved_search/medikace_mash.csv"):
    medikace_mash = pd.read_csv("saved_search/medikace_mash.csv")
    medikace_mash.index = medikace_mash["Unnamed: 0"]
    medikace_mash.drop(["Unnamed: 0"], axis=1, inplace=True)
    medikace_mash["search"] = medikace_mash["search"].apply(from_string_to_list)
else:
    medikace_mash = search_db_mash_optimized(medikace)
    medikace_mash.to_csv("saved_search/medikace_mash.csv")  
    
if os.path.isfile("saved_search/ne_medikace_mash.csv"):
    ne_medikace_mash = pd.read_csv("saved_search/ne_medikace_mash.csv")
    ne_medikace_mash.index = ne_medikace_mash["Unnamed: 0"]
    ne_medikace_mash.drop(["Unnamed: 0"], axis=1, inplace=True)
    ne_medikace_mash["search"] = ne_medikace_mash["search"].apply(from_string_to_list)
else:
    ne_medikace_mash = search_db_mash_optimized(ne_medikace)
    ne_medikace_mash.to_csv("saved_search/ne_medikace_mash.csv")    
    
if os.path.isfile("saved_search/symptom_mash.csv"):
    symptom_mash = pd.read_csv("saved_search/symptom_mash.csv")
    symptom_mash.index = symptom_mash["Unnamed: 0"]
    symptom_mash.drop(["Unnamed: 0"], axis=1, inplace=True)
    symptom_mash["search"] = symptom_mash["search"].apply(from_string_to_list)
else:
    symptom_mash = search_db_mash_optimized(symptom)
    symptom_mash.to_csv("saved_search/symptom_mash.csv")
    
if os.path.isfile("saved_search/ne_symptom_mash.csv"):
    ne_symptom_mash = pd.read_csv("saved_search/ne_symptom_mash.csv")
    ne_symptom_mash.index = ne_symptom_mash["Unnamed: 0"]
    ne_symptom_mash.drop(["Unnamed: 0"], axis=1, inplace=True)
    ne_symptom_mash["search"] = ne_symptom_mash["search"].apply(from_string_to_list)
else:
    ne_symptom_mash = search_db_mash_optimized(ne_symptom)
    ne_symptom_mash.to_csv("saved_search/ne_symptom_mash.csv")
    
if os.path.isfile("saved_search/procedura_mash.csv"):
    procedura_mash = pd.read_csv("saved_search/procedura_mash.csv")
    procedura_mash.index = procedura_mash["Unnamed: 0"]
    procedura_mash.drop(["Unnamed: 0"], axis=1, inplace=True)
    procedura_mash["search"] = procedura_mash["search"].apply(from_string_to_list)
else:
    procedura_mash = search_db_mash_optimized(procedura)
    procedura_mash.to_csv("saved_search/procedura_mash.csv")
    
data_mash = pd.concat([ne_os_anamneza_mash, ne_medikace_mash, ne_symptom_mash, symptom_mash, os_anamneza_mash, medikace_mash, procedura_mash])
data_mash = data_mash.sort_index()
    
    
print("osobní anamnéza:")    
print_stats(os_anamneza_mash["search"])
print("\nNE osobní anamnéza:")
print_stats(ne_os_anamneza_mash["search"])
print("\n\nmedikace:")
print_stats(medikace_mash["search"])
print("\nNE medikace:")
print_stats(ne_medikace_mash["search"])
print("\n\nsymptom:")
print_stats(symptom_mash["search"])
print("\nNE symptom:")
print_stats(ne_symptom_mash["search"])
print("\n\nprocedura:")
print_stats(procedura_mash["search"])

print("\n\nAll Data:")
print_stats(data_mash["search"])

osobní anamnéza:
Number of empty: 23 (10.74766355140187 %)
Mean from number of matches: 20.83177570093458
Median from number of matches: 19.5
Maximal of matches: 86

NE osobní anamnéza:
Number of empty: 4 (6.557377049180328 %)
Mean from number of matches: 20.852459016393443
Median from number of matches: 25.0
Maximal of matches: 75


medikace:
Number of empty: 74 (24.262295081967213 %)
Mean from number of matches: 14.177049180327868
Median from number of matches: 7.0
Maximal of matches: 82

NE medikace:
Number of empty: 1 (6.25 %)
Mean from number of matches: 19.3125
Median from number of matches: 15.5
Maximal of matches: 50


symptom:
Number of empty: 36 (6.132879045996593 %)
Mean from number of matches: 23.340715502555366
Median from number of matches: 22.0
Maximal of matches: 125

NE symptom:
Number of empty: 113 (11.03515625 %)
Mean from number of matches: 22.505859375
Median from number of matches: 25.0
Maximal of matches: 138


procedura:
Number of empty: 47 (7.885906040268456 %)

### Not assigned
If we look at the random sample of 10 texts, which are not assigned, then we can see that in five of them there is typographical mistake (*"nejsou zn.plicní hpertenze"* = *"nejsou zn. plicní hypertenze"*, *"kumulce a nehomogenity"* = *"kumulace a nehomogenita"*, *"ceriucal"* = *"cerucal"*, *"paitace"* = *"palpitace"*, *"mamily klidné"* = ?). Others five are correct medical term, but in some non-typical grammatical form.

If we try to improve them we get 50 % improvement.

In [107]:
# empty_sample = data_mash[data_mash["search"].apply(lambda x: len(x) == 0)].sample(10, random_state=42)

# Because of my mistake (I had worser clean_db), the code above generate different sample than I have worked with.
# So I have to create the sample by hand:
empty_sample = data_mash.loc[[878, 91, 5240, 3728, 1125, 2479, 4981, 1134, 5089, 1129]]

empty_sample["text"][878] = "nejsou zn. plicní hypertenze"
empty_sample["text"][91] = "hormostenické"
empty_sample["text"][5240] = "kumulace a nehomogenita"
empty_sample["text"][3728] = "biopsie"
empty_sample["text"][1125] = "chemobioterapie"
empty_sample["text"][2479] = "dysmorfické"
empty_sample["text"][4981] = "anikterické"
empty_sample["text"][1134] = "cerucal"
empty_sample["text"][5089] = "palpitace"
empty_sample["text"][1129] = "mamily klidné"

empty_sample = search_db_mash_optimized(empty_sample)
empty_sample

,text,search
Unnamed: 0,,
878,nejsou zn. plicní hypertenze,"[(C0020542, Pulmonary Hypertension), (C0152171..."
91,hormostenické,[]
5240,kumulace a nehomogenita,[]
3728,biopsie,"[(C0005558, Biopsy), (C0220797, biopsy charact..."
1125,chemobioterapie,[]
2479,dysmorfické,"[(C0005887, Body Dysmorphic Disorders)]"
4981,anikterické,[]
1134,cerucal,"[(C0701450, Cerucal)]"
5089,palpitace,"[(C0030252, Palpitations), (C0549267, Palpitat..."


There is one mistake which we can correct automaticly and it is not having space after punctuation mark. We can see that if we have space after punctuation then it find something, else it didn't.

We can see that there is around 150 examples of this mistakes.

In [108]:
print(len(search_with_inclusion("zn. plicní", mash_search)))
print(len(search_with_inclusion("zn.plicní", mash_search)))

50
0


In [109]:
def is_space_after_punc(string):
    punctuation = [".", ",", "!", "?", ":", ";", "+"]
    for i in range(len(string) - 1):
        if string[i] in punctuation and string[i+1] != " " and string[i+1] not in punctuation:
            return False
 
    return True

def insert_space_after_punc(string):
    punctuation = [".", ",", "!", "?", ":", ";", "+"]
    for i in range(len(string) - 1):
        if string[i] in punctuation and string[i+1] != " " and string[i+1] not in punctuation:
            string = string[:i+1] + " " + string[i+1:]
 
    return string

no_space = data[~data["text"].apply(is_space_after_punc)]
len(no_space)

155

In [110]:
data_mash["label"] = "N/A"
for i in data_mash.index:
    data_mash["label"][i] = data["label"][i]

if os.path.isfile("saved_search/data_mash_inserted_space.csv"):
    inserted_space_data_mash = pd.read_csv("saved_search/data_mash_inserted_space.csv")
    inserted_space_data_mash.index = inserted_space_data_mash["Unnamed: 0"]
    inserted_space_data_mash.drop(["Unnamed: 0"], axis=1, inplace=True)
    inserted_space_data_mash["search"] = inserted_space_data_mash["search"].apply(from_string_to_list)
else:
    inserted_space_data_mash = data_mash.copy()
    no_space["text"] = no_space["text"].apply(insert_space_after_punc)
    for i in no_space.index:
        inserted_space_data_mash["search"][i] = search_with_inclusion(no_space["text"][i], mash_search)
        inserted_space_data_mash["text"][i] = no_space["text"][i]
    inserted_space_data_mash.to_csv("saved_search/data_mash_inserted_space.csv")    
        
print_stats(data_mash["search"])
print()
print_stats(inserted_space_data_mash["search"])

Number of empty: 298 (10.631466286122013 %)
Mean from number of matches: 21.931858722797003
Median from number of matches: 25.0
Maximal of matches: 138

Number of empty: 256 (9.133071708883339 %)
Mean from number of matches: 22.383517659650373
Median from number of matches: 25.0
Maximal of matches: 138


Thanks to this upgrade we improved search by find 40 new matches.

## Linking to CZ Mash through Medvik 

In [111]:
import xml.etree.ElementTree as elt
import re

content = elt.parse('databaze/MeSH2023_Marc21_Alma.xml').getroot()

In [115]:
def patternize(string):
    result = []
    for i in string:
        if i in '<([{\\^-=$!|]})?*+.>]':
            result.append("\\" + i)
        else:
            result.append(i)
    return "".join(result)


def medvik_search(string, init_pattern, last_pattern):
    result = []
    pattern = re.compile(f"{init_pattern}{patternize(string)}{last_pattern}",re.IGNORECASE)
    
    for child in content:
        for subchild in child.iter("{http://www.loc.gov/MARC21/slim}subfield"):
            if subchild.text and pattern.match(subchild.text) is not None:
                try:
                    code = [i for i in child.findall("{http://www.loc.gov/MARC21/slim}controlfield") if i.attrib["tag"] == "001" ][0].text
                    name = [i for i in child.findall("{http://www.loc.gov/MARC21/slim}datafield") if i.attrib["tag"] == "150" ][0][0].text
                    result.append((code, name))
                    break
                except IndexError:
                    break                
    return result


def medvik_search_match(string):
    return medvik_search(string, ".*", ".*")


def medvik_search_exact(string):              
    return medvik_search(string, "^", "$")


def medvik_search_words(string):      
    return medvik_search(string, ".* ", " .*")


def medvik_search_combined(string):
    result = medvik_search(string, "^", "$")
    if len(result) == 0:
        result = medvik_search(string, ".* ", " .*")
    if len(result) == 0:
        result = medvik_search(string, ".*", ".*")
    
    return result


def search_db_medvik_match(db):
    return search_db(db, lambda x: search_with_inclusion(x, medvik_search_match, output_state=True))


def search_db_medvik_exact(db):
    return search_db(db, lambda x: search_with_inclusion(x, medvik_search_exact, output_state=True))


def search_db_medvik_words(db):
    return search_db(db, lambda x: search_with_inclusion(x, medvik_search_words, output_state=True))


def search_db_medvik_combined(db):
    return search_db(db, lambda x: search_with_inclusion(x, medvik_search_combined, output_state=True))

In [117]:
if os.path.isfile("saved_search/data_mash_medvik.csv"):
    data_mash_medvik = pd.read_csv("saved_search/data_mash_medvik.csv")
    data_mash_medvik.index = data_mash_medvik["Unnamed: 0"]
    data_mash_medvik.drop(["Unnamed: 0"], axis=1, inplace=True)
    data_mash_medvik["search"] = data_mash_medvik["search"].apply(from_string_to_list)
    data_mash_medvik["search_medvik"] = data_mash_medvik["search_medvik"].apply(from_string_to_list)
    data_mash_medvik = data_mash_medvik.rename(columns={"search": "search_mash"})
else:
    count = 0
    data_mash_medvik = data_mash_medvik.rename(columns={"search": "search_mash"})
    data_mash_medvik = search_db_medvik_match(inserted_space_data_mash)
    data_mash_medvik.to_csv("saved_search/data_mash_medvik.csv")
    data_mash_medvik = data_mash_medvik.rename(columns={"search": "search_medvik"})
    
print_stats(data_mash_medvik["search_mash"])
print()
print_stats(data_mash_medvik["search_medvik"])

Number of empty: 256 (9.133071708883339 %)
Mean from number of matches: 22.383517659650373
Median from number of matches: 25.0
Maximal of matches: 138

Number of empty: 270 (9.632536567962898 %)
Mean from number of matches: 1670.0117731002497
Median from number of matches: 28.0
Maximal of matches: 43684


In [118]:
print(data_mash_medvik["search_medvik"].apply(lambda x: len(x)).sample(10, random_state=42))

print()
temp = data_mash_medvik["search_medvik"].apply(lambda x: len(x))
len(temp[temp > 100])

Unnamed: 0
1562     1904
1469     1720
2438        0
5068    27715
2433     1722
894        94
212         9
1221    10724
4394        0
5457      112
Name: search_medvik, dtype: int64



1017

We can see, that some examples are working properly. On the other hand we have really lots of samples whose lenght grows exponentially.

For this reasons it might be better to use exact match instead of contains match.

In [119]:
test_data = data_mash.sample(20, random_state=42)

if os.path.isfile("saved_search/test_medvik.csv"):
    test_data = pd.read_csv("saved_search/test_medvik.csv")
    test_data["search_match"] = test_data["search_match"].apply(from_string_to_list)
    test_data["search_exact"] = test_data["search_exact"].apply(from_string_to_list)
    test_data["search_words"] = test_data["search_words"].apply(from_string_to_list)
    test_data["search_combined"] = test_data["search_combined"].apply(from_string_to_list)
else:
    count = 0
    test_data["search_match"] = search_db_medvik_match(test_data)["search"]
    test_data["search_exact"] = search_db_medvik_exact(test_data)["search"]
    test_data["search_words"] = search_db_medvik_words(test_data)["search"]
    test_data["search_combined"] = search_db_medvik_combined(test_data)["search"]
    test_data.to_csv("saved_search/test_medvik.csv")
    
print("Contains match")
print_stats(test_data["search_match"])
print("\nWords match")
print_stats(test_data["search_words"])
print("\nExact match")
print_stats(test_data["search_exact"])
print("\nCombined match")
print_stats(test_data["search_combined"])

Contains match
Number of empty: 3 (15.0 %)
Mean from number of matches: 2265.2
Median from number of matches: 15.5
Maximal of matches: 27715

Words match
Number of empty: 6 (30.0 %)
Mean from number of matches: 194.0
Median from number of matches: 5.5
Maximal of matches: 1918

Exact match
Number of empty: 11 (55.00000000000001 %)
Mean from number of matches: 3.05
Median from number of matches: 0.0
Maximal of matches: 22

Combined match
Number of empty: 3 (15.0 %)
Mean from number of matches: 261.95
Median from number of matches: 6.5
Maximal of matches: 1904


We can see that using exact match we get rid of the long matches but it have quite low success rate. Using words match is something in the middle (not good in both ways).

As last option we used combined match (first try exact, if don't success then words, then only match). This seems as the best methods (this doesn't create too large lists and has the same number of empty matches as contains match) 

In [120]:
if os.path.isfile("saved_search/data_mash_medvik_combined.csv"):
    data_mash_medvik_combined = pd.read_csv("saved_search/data_mash_medvik_combined.csv")
    data_mash_medvik_combined.index = data_mash_medvik_combined["Unnamed: 0"]
    data_mash_medvik_combined.drop(["Unnamed: 0"], axis=1, inplace=True)
    data_mash_medvik_combined["search_mash"] = data_mash_medvik_combined["search_mash"].apply(from_string_to_list)
    data_mash_medvik_combined["search_medvik"] = data_mash_medvik_combined["search_medvik"].apply(from_string_to_list)
else:
    count = 0
    data_mash_medvik_combined = inserted_space_data_mash.rename(columns={"search": "search_mash"})
    data_mash_medvik_combined = search_db_medvik_combined(data_mash_medvik_combined)
    data_mash_medvik_combined = data_mash_medvik_combined.rename(columns={"search": "search_medvik"})
    data_mash_medvik_combined.to_csv("saved_search/data_mash_medvik_combined.csv")
    
    
print_stats(data_mash_medvik_combined["search_medvik"])

Number of empty: 270 (9.632536567962898 %)
Mean from number of matches: 488.3999286478773
Median from number of matches: 7.0
Maximal of matches: 28651


### Duplicates
It is possible to get duplicates in list of matches, when getting the same match from two different words from text.

So I will remove them and observe how it change results.

In [121]:
len(inserted_space_data_mash[~inserted_space_data_mash["search"].apply(lambda x: len(set(x)) == len(x))])

22

In [122]:
def remove_dup_preserve_order(l):
    seen = set()
    seen_add = seen.add
    return [x for x in l if not (x in seen or seen_add(x))]

print("Before removing duplicates:")
print("Mash search")
print_stats(data_mash_medvik["search_mash"])
print("\nMedvik contains search")
print_stats(data_mash_medvik["search_medvik"])
print("\nMedvik combined search")
print_stats(data_mash_medvik_combined["search_medvik"])

data_mash_medvik["search_mash"] = data_mash_medvik["search_mash"].apply(remove_dup_preserve_order)
data_mash_medvik["search_medvik"] = data_mash_medvik["search_medvik"].apply(remove_dup_preserve_order)
data_mash_medvik_combined["search_mash"] = data_mash_medvik["search_mash"]
data_mash_medvik_combined["search_medvik"] = data_mash_medvik_combined["search_medvik"].apply(remove_dup_preserve_order)

print("\n\nAfter removing duplicates:")
print("Mash search")
print_stats(data_mash_medvik["search_mash"])
print("\nMedvik contains search")
print_stats(data_mash_medvik["search_medvik"])
print("\nMedvik combined search")
print_stats(data_mash_medvik_combined["search_medvik"])

Before removing duplicates:
Mash search
Number of empty: 256 (9.133071708883339 %)
Mean from number of matches: 22.383517659650373
Median from number of matches: 25.0
Maximal of matches: 138

Medvik contains search
Number of empty: 270 (9.632536567962898 %)
Mean from number of matches: 1670.0117731002497
Median from number of matches: 28.0
Maximal of matches: 43684

Medvik combined search
Number of empty: 270 (9.632536567962898 %)
Mean from number of matches: 488.3999286478773
Median from number of matches: 7.0
Maximal of matches: 28651


After removing duplicates:
Mash search
Number of empty: 256 (9.133071708883339 %)
Mean from number of matches: 22.33927934356047
Median from number of matches: 25.0
Maximal of matches: 138

Medvik contains search
Number of empty: 270 (9.632536567962898 %)
Mean from number of matches: 1625.7213699607564
Median from number of matches: 28.0
Maximal of matches: 29327

Medvik combined search
Number of empty: 270 (9.632536567962898 %)
Mean from number of ma